In [2]:
import requests
import re
import pandas as pd
import numpy as np
url = 'http://www.legco.gov.hk/yr14-15/chinese/counmtg/voting/cm_vote_20141022.xml'
page = requests.get(url).content

In [28]:
motion = re.search(r'<motion-en>(.+)</motion-en>',page).group(1)
names = re.findall(r'name-en="(.+?)" ',page)
vote = re.findall(r'name-en="(.+?)" ',page)
mover_pt = page.find('/mover-en')
page_no_dup = page[mover_pt:]

In [37]:
def get_vote(name):
    pt = page.find(name)
    vote = re.search(r'<vote>(.+?)</vote>',page[pt:]).group(1)
    return vote

In [25]:
df = pd.DataFrame(names, columns=['name'])

In [39]:
df[motion] = df['name'].apply(get_vote)
df.head()
   

,name,MARRIAGE (AMENDMENT) BILL 2014 - SECOND READING
0,TSANG Yok-sing,Present
1,Albert HO,No
2,LEE Cheuk-yan,No
3,James TO,Absent
4,CHAN Kam-lam,No


In [27]:
page.find('/mover-en')

522

In [36]:
pt = page.find('James TO')
print pt
re.search(r'<vote>(.+?)</vote>',page[pt:]).group(1)

1205


'Absent'

In [42]:
url = 'http://www.legco.gov.hk/yr14-15/chinese/counmtg/voting/cm_vote_20141021.xml'
requests.get(url).status_code

404

In [113]:
def get_vote(name, page):
    pt = page.find(name)
    vote = re.search(r'<vote>(.+?)</vote>',page[pt:]).group(1)
    #print name, vote
    return vote


yr_dic = {'12-13':[2012,2013],
          '13-14':[2013,2014],
          '14-15':[2014,2015],
          '15-16':[2015,2016]}
months = range(1,13)
days = range(1,32)
df = ''

for (period,years) in yr_dic.items():
    for year in years:
        y = str(year)
        for month in months:
            m = str(month)
            if month<10:
                m = '0' + str(month)
            for day in days:
                d = str(day)
                if day<10:
                    d = '0' + str(day)
                target = y+m+d
                url = 'http://www.legco.gov.hk/yr'+period+'/chinese/counmtg/voting/cm_vote_'+target+'.xml'
                req = requests.get(url)
                code = req.status_code
                if code == 200:
                    #print target + 'yes'
                    page = req.content
                    moves = re.findall(r'<motion-en>.+?</motion-en>',page)
                    move_pts = [page.find(move) for move in moves]
                    for mp in move_pts:
                        temp_page = page[mp:]
                        motion = re.search(r'<motion-en>(.+?)</motion-en>',temp_page).group(1)
                        mover_pt = temp_page.find('/mover-en')
                        page_no_dup = temp_page[mover_pt:]
                        if type(df) != str:
                            df[motion] = df['name'].apply(lambda x: get_vote(x,page_no_dup))
                        else:
                            names = re.findall(r'name-en="(.+?)" ',temp_page)
                            df = pd.DataFrame(np.unique(names), columns=['name'])
                            df[motion] = df['name'].apply(lambda x: get_vote(x,page_no_dup))
                #print df
        #print target + 'no'

AttributeError: 'NoneType' object has no attribute 'group'

In [107]:
df.shape

(70, 80)

In [56]:
len(re.findall(r'<motion-en>(.+?)</motion-en>',page))

8

In [60]:
x = ['a','b','b','c']
np.unique(x)


array(['a', 'b', 'c'], 
      dtype='|S1')

In [102]:
df.head()

,name,MOTION FOR THE ADJOURNMENT OF THE COUNCIL UNDER RULE 16(2) OF THE RULES OF PROCEDURE,MARRIAGE (AMENDMENT) BILL 2014 - SECOND READING,MOTION UNDER RULE 40(1) OF THE RULES OF PROCEDURE THAT THE DEBATE ON THE PROPOSED RESOLUTION UNDER SECTION 54A OF THE INTERPRETATION AND GENERAL CLAUSES ORDINANCE BE NOW ADJOURNED,AMENDMENT BY HON CHAN CHI-CHUEN TO THE PROPOSED RESOLUTION MOVED BY THE SECRETARY FOR COMMERCE AND ECONOMIC DEVELOPMENT UNDER SECTION 54A OF THE INTERPRETATION AND GENERAL CLAUSES ORDINANCE (THE 1ST AMENDMENT),MOTION TO SHORTEN DURATION OF DIVISION BELL,AMENDMENT BY HON CHAN CHI-CHUEN TO THE PROPOSED RESOLUTION MOVED BY THE SECRETARY FOR COMMERCE AND ECONOMIC DEVELOPMENT UNDER SECTION 54A OF THE INTERPRETATION AND GENERAL CLAUSES ORDINANCE (THE 2ND AMENDMENT),AMENDMENT BY HON ALBERT CHAN TO THE PROPOSED RESOLUTION MOVED BY THE SECRETARY FOR COMMERCE AND ECONOMIC DEVELOPMENT UNDER SECTION 54A OF THE INTERPRETATION AND GENERAL CLAUSES ORDINANCE (THE 3RD AMENDMENT),AMENDMENT BY HON ALBERT CHAN TO THE PROPOSED RESOLUTION MOVED BY THE SECRETARY FOR COMMERCE AND ECONOMIC DEVELOPMENT UNDER SECTION 54A OF THE INTERPRETATION AND GENERAL CLAUSES ORDINANCE (THE 4TH AMENDMENT),AMENDMENT BY HON CHAN CHI-CHUEN TO THE PROPOSED RESOLUTION MOVED BY THE SECRETARY FOR COMMERCE AND ECONOMIC DEVELOPMENT UNDER SECTION 54A OF THE INTERPRETATION AND GENERAL CLAUSES ORDINANCE (THE 5TH AMENDMENT),...,AMENDMENT BY HON TANG KA-PIU TO HON GARY FAN’S MOTION ON “ADHERING TO THE NEED TO ‘PUT HONG KONG PEOPLE FIRST’ IN FORMULATING POLICIES”,MOTION ON “ADHERING TO THE NEED TO ‘PUT HONG KONG PEOPLE FIRST’ IN FORMULATING POLICIES”,ADMINISTRATION OF JUSTICE (MISCELLANEOUS PROVISIONS) BILL 2014 - THIRD READING,EMPLOYMENT (AMENDMENT) BILL 2014 - SECOND READING,EMPLOYMENT (AMENDMENT) BILL 2014 - COMMITTEE STAGE - THE SECRETARY FOR LABOUR AND WELFARE’S AMENDMENTS TO CLAUSE 6,EMPLOYMENT (AMENDMENT) BILL 2014 - COMMITTEE STAGE - HON KENNETH LEUNG’S 1ST AMENDMENT TO CLAUSE 6,EMPLOYMENT (AMENDMENT) BILL 2014 - COMMITTEE STAGE - HON LEE CHEUK-YAN’S 1ST AMENDMENT TO CLAUSE 6,EMPLOYMENT (AMENDMENT) BILL 2014 - COMMITTEE STAGE - HON KENNETH LEUNG’S 2ND AMENDMENT TO CLAUSE 6,EMPLOYMENT (AMENDMENT) BILL 2014 - COMMITTEE STAGE - HON LEE CHEUK-YAN’S 2ND AMENDMENT TO CLAUSE 6,EMPLOYMENT (AMENDMENT) BILL 2014 - THIRD READING
0,Abraham SHEK,Absent,No,Absent,No,Yes,No,No,No,No,...,Absent,Absent,Absent,Yes,Yes,Absent,Absent,Absent,Absent,Yes
1,Alan LEONG,Yes,No,Absent,Absent,Absent,Absent,Absent,Absent,Absent,...,No,Abstain,No,Absent,Absent,Absent,Yes,Absent,Yes,Yes
2,Albert CHAN,Yes,No,Yes,Yes,No,Yes,Yes,Yes,Yes,...,No,Abstain,Yes,Absent,Yes,Absent,Yes,Yes,Yes,Yes
3,Albert HO,Absent,No,Yes,Abstain,Absent,No,No,No,Absent,...,No,Abstain,Absent,Yes,Yes,Absent,Yes,Yes,Yes,Yes
4,Alice MAK,No,Yes,No,No,Absent,Absent,Absent,Absent,Absent,...,Yes,No,Yes,Yes,Yes,Present,Present,Present,Present,Yes


In [90]:
pd.Series(df.iloc[1]).value_counts()

Absent        49
Yes           18
No             6
Abstain        5
Alan LEONG     1
Name: 1, dtype: int64

In [103]:
tdf = df.transpose()

In [104]:
for i in range(tdf.shape[1]):
    print tdf.iloc[0,i]
    print tdf.iloc[1:,i].value_counts()

Abraham SHEK
No        37
Absent    33
Yes        9
Name: 0, dtype: int64
Alan LEONG
Absent     49
Yes        19
No          6
Abstain     5
Name: 1, dtype: int64
Albert CHAN
Yes        55
Absent     13
No          7
Abstain     4
Name: 2, dtype: int64
Albert HO
Absent     51
Yes        16
No          8
Abstain     4
Name: 3, dtype: int64
Alice MAK
Absent     32
Yes        31
No         12
Present     4
Name: 4, dtype: int64
Andrew LEUNG
No         46
Yes        16
Present    13
Abstain     2
Absent      2
Name: 5, dtype: int64
CHAN Chi-chuen
Yes        62
No          9
Absent      5
Abstain     3
Name: 6, dtype: int64
CHAN Hak-kan
No         44
Absent     18
Yes        14
Abstain     3
Name: 7, dtype: int64
CHAN Han-pan
No         44
Yes        27
Absent      5
Abstain     3
Name: 8, dtype: int64
CHAN Kam-lam
No         45
Yes        29
Abstain     3
Absent      2
Name: 9, dtype: int64
CHAN Kin-por
No         45
Yes        28
Absent      4
Abstain     2
Name: 10, dtype: int64
CHAN Yue

In [95]:
tdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79 entries, name to EMPLOYMENT (AMENDMENT) BILL 2014 - THIRD READING
Data columns (total 70 columns):
0     79 non-null object
1     79 non-null object
2     79 non-null object
3     79 non-null object
4     79 non-null object
5     79 non-null object
6     79 non-null object
7     79 non-null object
8     79 non-null object
9     79 non-null object
10    79 non-null object
11    79 non-null object
12    79 non-null object
13    79 non-null object
14    79 non-null object
15    79 non-null object
16    79 non-null object
17    79 non-null object
18    79 non-null object
19    79 non-null object
20    79 non-null object
21    79 non-null object
22    79 non-null object
23    79 non-null object
24    79 non-null object
25    79 non-null object
26    79 non-null object
27    79 non-null object
28    79 non-null object
29    79 non-null object
30    79 non-null object
31    79 non-null object
32    79 non-null object
33    79 non-null object
34 

In [98]:
df.iloc[:,:2]

,name,MARRIAGE (AMENDMENT) BILL 2014 - SECOND READING
0,Abraham SHEK,No
1,Alan LEONG,No
2,Albert CHAN,No
3,Albert HO,No
4,Alice MAK,Yes
5,Andrew LEUNG,No
6,CHAN Chi-chuen,No
7,CHAN Hak-kan,No
8,CHAN Han-pan,No
9,CHAN Kam-lam,No


In [100]:
df.columns

Index([u'name', u'MARRIAGE (AMENDMENT) BILL 2014 - SECOND READING',
       u'MOTION UNDER RULE 40(1) OF THE RULES OF PROCEDURE THAT THE DEBATE ON THE PROPOSED RESOLUTION UNDER SECTION 54A OF THE INTERPRETATION AND GENERAL CLAUSES ORDINANCE BE NOW ADJOURNED',
       u'AMENDMENT BY HON CHAN CHI-CHUEN TO THE PROPOSED RESOLUTION MOVED BY THE SECRETARY FOR COMMERCE AND ECONOMIC DEVELOPMENT UNDER SECTION 54A OF THE INTERPRETATION AND GENERAL CLAUSES ORDINANCE (THE 1ST AMENDMENT)',
       u'MOTION TO SHORTEN DURATION OF DIVISION BELL',
       u'AMENDMENT BY HON CHAN CHI-CHUEN TO THE PROPOSED RESOLUTION MOVED BY THE SECRETARY FOR COMMERCE AND ECONOMIC DEVELOPMENT UNDER SECTION 54A OF THE INTERPRETATION AND GENERAL CLAUSES ORDINANCE (THE 2ND AMENDMENT)',
       u'AMENDMENT BY HON ALBERT CHAN TO THE PROPOSED RESOLUTION MOVED BY THE SECRETARY FOR COMMERCE AND ECONOMIC DEVELOPMENT UNDER SECTION 54A OF THE INTERPRETATION AND GENERAL CLAUSES ORDINANCE (THE 3RD AMENDMENT)',
       u'AMENDMENT BY HON 

In [111]:
yr_dic = {'12-13':[2012,2013],
          '13-14':[2013,2014],
          '14-15':[2014,2015],
          '15-16':[2015,2016]}
yr_dic.items()

[('15-16', [2015, 2016]),
 ('14-15', [2014, 2015]),
 ('13-14', [2013, 2014]),
 ('12-13', [2012, 2013])]